# My Music 

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

from mp3id import mp3ID
from musicBase import musicBase
from musicFinder import musicFinder
from musicPath import pathBasics
from timeUtils import clock, elapsed
from discogsBase import discogs
from multiArtist import multiArtist
from time import sleep
#from discogs import discogs

## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-09-05 20:50:46.295763


# Test

In [ ]:
ifile = "/Volumes/Music/iTunes MixTape/iTunes Media/Music/24 Shots/01 Intro (Power Of The Dollar).mp3"
isFile(ifile)

In [ ]:
m = mp3ID(ifile)
m.getInfo()

In [ ]:
pb  = pathBasics(debug=False)
pbc = pb.getPaths(ifile)
pbc.getDict()

# Find All Music

In [12]:
%load_ext autoreload
%autoreload
mf = musicFinder(debug=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
## Run This To Find Music Paths
mf.findMusic()

Checking /Volumes/Music/iTunes Classical...   	Found 3391 files.
Processed 3391 paths.
Saving data to /Users/tgadfort/Documents/code/music/db/Music-/Volumes/Music/iTunes Classical.p


FileNotFoundError: [Errno 2] No such file or directory: '/Users/tgadfort/Documents/code/music/db/Music-/Volumes/Music/iTunes Classical.p'

In [10]:
mf.findMusicDBPaths()

In [ ]:
mf.findMusicDBTags()

In [3]:
%load_ext autoreload
%autoreload
disc = discogs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


In [ ]:
#mm.findMusic()
#files = mm.getMusicDBs()

In [ ]:
#mm.analyzeMusicDBPaths()
#mm.analyzeMusicDBTags()

In [4]:
from searchUtils import findDirsPattern, findWalkExt
from fileUtils import getBaseFilename, getFileBasics
from fsUtils import setFile
from ioUtils import saveFile, getFile
from timeUtils import clock, elapsed
from musicPath import pathBasics


##############################################################################################################################
# My Music
##############################################################################################################################
class classicalMusicAnalyzer(musicBase):
    def __init__(self, debug=False):
        self.name = "cma"
        musicBase.__init__(self, debug=debug)

In [5]:
mca = classicalMusicAnalyzer()

# Master Discog DB

In [11]:
from pandas import Series, DataFrame

def realName(x):
    if x is None:
        return [None,-1]
    
    lenx = len(x)
    if len(x) < 1:
        return [x,-1]

        

    if x[-1] != ")":
        return [x, None]
    

    if lenx >=5:
        if x[-3] == "(":
            try:
                num = int(x[-2:-1])
                val = x[:-3].strip()
                return [val, num]
            except:
                return [x, None]
            
    if lenx >= 6:
        if x[-4] == "(":
            try:
                num = int(x[-3:-1])
                val = x[:-4].strip()
                return [val, num]
            except:
                return [x, None]
            
    if lenx >= 7:
        if x[-4] == "(":
            try:
                num = int(x[-3:-1])
                val = x[:-4].strip()
                return [val, num]
            except:
                return [x, None]

    return [x, None]
            

def getAlbumNames(x):
    if isinstance(x, dict):
        return list(x.values())
    else:
        return []
    
    
def splitMetaData(x):
    retval = {}
    if isinstance(x, dict):
        for k,v in x.items():
            retval[k] = [z[0] for z in v.most_common(3)]
    else:
        retval = None
    return retval

def createArtistAlbumsDB(disc):
    print("Creating ArtistAlbums DB")
    
    print("  Loading ArtistID Data")
    artistIDtoName  = disc.getArtistIDToNameData()
    artistIDtoRefs  = disc.getArtistIDToRefData()
    
    print("  Loading AlbumID Data")
    albumIDtoName   = disc.getAlbumIDToNameData()
    albumIDtoRef    = disc.getAlbumIDToRefData()

    print("  Loading ArtistID <-> AlbumID Data")
    artistIDtoAlbumNames = disc.getArtistIDAlbumNames()
    artistIDtoAlbumIDs   = disc.getArtistIDAlbumIDs()

    print("  Loading Artist MetaData")
    artistMetaData = disc.getAlbumArtistMetaData()


    sArtistToRef  = Series(artistIDtoRefs)
    sArtistToName = Series(artistIDtoName)
    sAlbumToRef   = Series(albumIDtoRef)
    sAlbumToName  = Series(albumIDtoName)

    sArtistToAlbums = Series(artistIDtoAlbumIDs)
    sArtistToAlbumNames = Series(artistIDtoAlbumNames)
    sArtistAlbums = Series([dict(zip(x, y)) for x,y in list(zip(sArtistToAlbums.values, sArtistToAlbumNames))], index=sArtistToAlbums.index)

    sArtistMetaData = Series(artistMetaData)
    sArtistMetaData = sArtistMetaData.apply(splitMetaData)
    
    print("  Creating Pandas DataFrame for {0} Artists".format(sArtistToRef.shape[0]))
    cols = ["Ref"]
    discdf = DataFrame(sArtistToRef)
    discdf.columns = cols
    discdf = discdf.join(DataFrame(sArtistToName))
    cols += ["Name"]
    discdf.columns = cols
    tmp = DataFrame(DataFrame(sArtistMetaData)[0].tolist())
    tmp.index = sArtistMetaData.index
    discdf = discdf.join(tmp)
    cols += ["Extra Artists", "Genres", "Styles"]
    discdf.columns = cols
    discdf = discdf.join(DataFrame(sArtistAlbums))
    cols += ["Albums Data"]
    discdf.columns = cols
    discdf["Known"] = True
    print("  DataFrame Shape is {0}".format(discdf.shape))    
    return discdf


def createArtistName(discdf):
    tmp = DataFrame(discdf["Name"].apply(realName).tolist())
    tmp.index = discdf.index
    discdf["Artist"]     = tmp[0]
    discdf["Artist Num"] = tmp[1]
    return discdf


def createAlbums(discdf):
    discdf["Albums"] = discdf["Albums Data"].apply(getAlbumNames)
    return discdf


def createCollectionsDB(disc):
    print("Creating Collections DB")

    print("  Loading Collection Data")
    colArtistIDtoName  = disc.getCollectionIDToNameData()
    colArtistIDtoRefs  = disc.getCollectionIDToRefData()
    colArtistReftoCnts = disc.getCollectionRefCountsData()

    sColArtistToRef  = Series(colArtistIDtoRefs)
    sColArtistToName = Series(colArtistIDtoName)
    sColArtistRefToCnts = Series(colArtistReftoCnts)

    print("  Creating Pandas DataFrame for {0} Artists".format(sColArtistToRef.shape[0]))
    cols = ["Ref"]
    coldiscdf = DataFrame(sColArtistToRef)
    coldiscdf.columns = cols
    coldiscdf = coldiscdf.join(DataFrame(sColArtistToName))
    cols += ["Name"]
    coldiscdf.columns = cols

    colrefdf = DataFrame(sColArtistRefToCnts)
    colrefdf.columns = ["Counts"]
    colrefdf.reset_index(inplace=True)
    colrefdf.columns = ["Ref", "Counts"]
    coldiscdf = coldiscdf.merge(colrefdf, on="Ref")
    coldiscdf.index = DataFrame(sColArtistToRef).index
    
    print("  DataFrame Shape is {0}".format(coldiscdf.shape))
    return coldiscdf


def mergeDBs(discdf, coldiscdf):
    print("  Merging AlbumArtists {0} and Collections {1} DBs".format(discdf.shape, coldiscdf.shape))
    musicdf = coldiscdf.merge(discdf, on=["Ref", "Name"], how='left')
    musicdf.index = coldiscdf.index
    print("  Merged DataFrame Shape is {0}".format(musicdf.shape))
    return musicdf

In [36]:
discdf    = createArtistAlbumsDB(disc)
discdf    = createArtistName(discdf)
#discdf    = createAlbums(discdf)
#coldiscdf = createCollectionsDB(disc)
#musicdf   = mergeDBs(discdf, coldiscdf)

#saveFile(idata=musicdf, ifile="musicDF.p")

Creating ArtistAlbums DB
  Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 8.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 10.3MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
  Loading AlbumID Data
Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 11.7MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 16.1MB.
Loading /Volumes/Music/Discog/db/AlbumIDToRef.p
  Loading ArtistID <-> AlbumID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDAlbumNames.p
  --> This file is 72.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDAlbumNames.p
Loading data from /Volumes/Music/Discog/db/ArtistIDAlbumIDs.p
  --> This file is 42.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDAlbumIDs.p
  Loading Artist MetaData
Loading data from /Volumes/Music/Discog/d

In [38]:
discdf    = createAlbums(discdf)
discdf

,Ref,Name,Extra Artists,Genres,Styles,Albums Data,Known,Artist,Artist Num,Albums
1000,/artist/1000-Dave-Clarke,Dave Clarke,[Dave Clarke],"[Electronic, Hip Hop, Non-Music]","[Techno, Electro, EBM]","{'856965': 'Fuse Presents Dave Clarke', '27412...",True,Dave Clarke,NaN,"[Fuse Presents Dave Clarke, Compass, Red. 1 (O..."
1000500,/artist/1000500-Club-Pulse,Club Pulse,[],[],[],{'1178875': 'Peak Controller'},True,Club Pulse,NaN,[Peak Controller]
100200,/artist/100200-Dike,Dike,[Dike],[Hip Hop],[Conscious],"{'2082992': 'Infinity EP (Instrumental)', '156...",True,Dike,NaN,"[Infinity EP (Instrumental), Der Wille ZÃ¤hlt ..."
1002000,/artist/1002000-Larry-Stokes,Larry Stokes,[],[],[],"{'124656': 'Blunted On Reality', '1002108': 'N...",True,Larry Stokes,NaN,"[Blunted On Reality, Nappy Heads, Are You Fait..."
1002600,/artist/1002600-Gidd-Sanchez,Gidd Sanchez,NaN,NaN,NaN,{'1180905': 'Quiero'},True,Gidd Sanchez,NaN,[Quiero]
1002900,/artist/1002900-John-Keding,John Keding,NaN,NaN,NaN,{'157011': 'Freedom Fields'},True,John Keding,NaN,[Freedom Fields]
1003900,/artist/1003900-Jason-Pancho,Jason Pancho,NaN,NaN,NaN,"{'157268': 'Untitled', '330170': 'Loca Motion ...",True,Jason Pancho,NaN,"[Untitled, Loca Motion / Pale Drifter, D.J. Cl..."
1005400,/artist/1005400-Gapeseed,Gapeseed,[Gapeseed],[Rock],"[Alternative Rock, Indie Rock, Math Rock]","{'4313892': 'Project 64', '3689834': 'Lo Cell'...",True,Gapeseed,NaN,"[Project 64, Lo Cell, Flanzer B/W Thunder Tong..."
1007400,/artist/1007400-The-Beauty-Shop,The Beauty Shop,[The Beauty Shop],[Rock],"[Country Rock, Indie Rock, Alternative Rock]","{'649563': 'Monster', '807093': 'Yr Money Or Y...",True,The Beauty Shop,NaN,"[Monster, Yr Money Or Yr Life, A Desperate Cry..."
1010100,/artist/1010100-The-Salads,The Salads,[The Salads],"[Rock, Reggae, Hip Hop]","[, Punk, Ska]","{'7858352': 'Who's That?', '7342091': 'Music E...",True,The Salads,NaN,"[Who's That?, Music Every Day, Fold A To B, Me..."


In [13]:
musicdf.head()

,Ref,Name,Counts,Extra Artists,Genres,Styles,Albums Data,Known,Artist,Artist Num,Albums
294746,/artist/294746-Richard-Wagner,Wagner,2975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4756306,/artist/4756306-Edgardo-Donato-Y-Su-Orquesta-T...,Edgardo Donato Y Su Orquesta Típica,8,[Edgardo Donato Y Su Orquesta Típica],"[Classical, Folk, World, & Country]",[],"{'12296603': 'El Camion / Carton Ligador', '57...",True,Edgardo Donato Y Su Orquesta Típica,NaN,"[El Camion / Carton Ligador, Buenos Aires Tiem..."
5224521,/artist/5224521-TrÃ­o-Cocomarola,TrÃ­o Cocomarola,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4775262,/artist/4775262-Damasio-Esquivel-y-Su-Conjunto...,Damasio Esquivel y Su Conjunto Guaraní,9,[Damasio Esquivel y Su Conjunto Guaraní],"[Latin, Folk, World, & Country]","[Polka, Chamamé]",{'5747330': 'Pajaro Campana / Ah! Me Corriente...,True,Damasio Esquivel y Su Conjunto Guaraní,NaN,[Pajaro Campana / Ah! Me Corrientes Pora ]
392779,/artist/392779-Carlos-Gardel,Carlos Gardel,606,[Carlos Gardel],"[Latin, Folk, World, & Country, Non-Music]","[Tango, Zamba, Ranchera]",{'1363649': 'Un Tropezón / Se Acabaron Los Ota...,True,Carlos Gardel,NaN,"[Un Tropezón / Se Acabaron Los Otarios, Serie ..."


In [15]:
AlbIDToName  = disc.getAlbumIDToNameData()
AlbIDToRef   = disc.getAlbumIDToRefData()
AlbIDToArtID = disc.getAlbumIDToArtistIDData()

Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 11.7MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 16.1MB.
Loading /Volumes/Music/Discog/db/AlbumIDToRef.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToArtistID.p
  --> This file is 7.6MB.
Loading /Volumes/Music/Discog/db/AlbumIDToArtistID.p


In [20]:
sAlbumToRef   = Series(AlbIDToRef)
sAlbumToName  = Series(AlbIDToName)

sAlbumToArtist = Series(AlbIDToArtID)
#sArtistAlbums = Series([dict(zip(x, y)) for x,y in list(zip(sArtistToAlbums.values, sArtistToAlbumNames))], index=sArtistToAlbums.index)

print("  Creating Pandas DataFrame for {0} Albums".format(sAlbumToRef.shape[0]))
cols = ["Ref"]
albdf = DataFrame(sAlbumToRef)
albdf.columns = cols
albdf = albdf.join(DataFrame(sAlbumToName))
cols += ["Name"]
albdf.columns = cols
albdf = albdf.join(DataFrame(sAlbumToArtist))
cols += ["Artist ID"]
albdf.columns = cols

  Creating Pandas DataFrame for 530359 Albums


# Music Analyzer (My Music)

In [91]:
from searchUtils import findDirsPattern, findWalkExt
from fileUtils import getBaseFilename, getFileBasics
from fsUtils import setFile
from ioUtils import saveFile, getFile
from timeUtils import clock, elapsed
from musicPath import pathBasics


##############################################################################################################################
# My Music
##############################################################################################################################
class musicAnalyzer(musicBase):
    def __init__(self, debug=False):
        self.name = "myanalyzer"
        musicBase.__init__(self, debug=debug)
        
        self.pb      = pathBasics()
        self.flists  = self.getMusicListFiles()
        self.tagDBs  = self.getMusicTagDBs()
        self.pathDBs = self.getMusicPathDBs()

In [92]:
ma = musicAnalyzer(debug=False)

Found 13 List Files
Found 12 DB Files
Found 12 DB Files


In [93]:
jzpath = ma.pathDBs[-2]
jztag  = ma.tagDBs[1]
jzpathdata = getFile(jzpath)
jztagdata  = getFile(jztag)

In [189]:
start, cmt = clock("Getting My Artists/Albums")

artistData = {}
albumData = {}
for pathfile in ma.pathDBs:
    pathdata = getFile(pathfile)
    for mp3,mp3data in pathdata.items():
        if artistData.get(mp3data['pbArtist']) is None:
            artistData[mp3data["pbArtist"]] = {}
        if artistData[mp3data["pbArtist"]].get(mp3data["pbAlbum"]) is None:
            artistData[mp3data["pbArtist"]][mp3data["pbAlbum"]] = []
        artistData[mp3data["pbArtist"]][mp3data["pbAlbum"]].append(mp3)

        if albumData.get(mp3data["pbAlbum"]) is None:
            albumData[mp3data["pbAlbum"]] = {}
        if albumData[mp3data["pbAlbum"]].get(mp3data["pbArtist"]) is None:
            albumData[mp3data["pbAlbum"]][mp3data["pbArtist"]] = []
        albumData[mp3data["pbAlbum"]][mp3data["pbArtist"]].append(mp3)
        
    print("Found {0} artists and {1} albums".format(len(artistData), len(albumData)))
    
elapsed(start, cmt)

Current Time is Tue Sep 03, 2019 21:01:36 for Getting My Artists/Albums
Found 3739 artists and 10245 albums
Found 3739 artists and 10245 albums
Found 3739 artists and 10247 albums
Found 4522 artists and 25351 albums
Found 7052 artists and 31574 albums
Found 7072 artists and 31588 albums
Found 7072 artists and 31588 albums
Found 7072 artists and 31588 albums
Found 7076 artists and 31591 albums
Found 7137 artists and 31853 albums
Found 7150 artists and 32143 albums
Found 7207 artists and 38874 albums
Current Time is Tue Sep 03, 2019 21:02:18 for Done with Getting My Artists/Albums
Process [{0}] took 42 seconds.


# Find Artists

In [95]:
ex = read_excel("/Users/tgadfort/Downloads/Artists-4.xlsx")
discogReplacements = dict(zip(ex["Discogs"], ex["Called"]))
renameReplacements = dict(zip(ex['Renamed'], ex['Actual']))

In [196]:
import copy
saveArtistDataPrev = copy.deepcopy(saveArtistData)

In [197]:
from searchUtils import findNearest

def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None

artists = list(discdf["Artist"])

saveArtistData = {"Odd": {}, "Match": {}, "Near": {}, "None": {}}

for artistName,artistAlbums in artistData.items():
    
    rename = None
    if renameReplacements.get(artistName) is not None:
        artistName = renameReplacements[artistName]
        rename = True
        
    
    artistName = normalize('NFC', artistName)

    
    keys = ["Artist", "Name"]
    done = False
    for key in keys:
        mdata = getMusicData(key, artistName)
        if isinstance(mdata, DataFrame):
            if key == "Artist":
                print('Match\t',mdata.shape,'\t',artistName)
                saveArtistData["Match"][artistName] = {"Rename": rename, "Discog": mdata, "Albums": artistAlbums}
            else:
                print('Odd\t',mdata.shape,'\t',artistName)
                saveArtistData["Odd"][artistName] = {"Rename": rename, "Discog": mdata, "Albums": artistAlbums}
            done = True
            break
    if done:
        continue

    cutoff=0.9
    if len(artistName) <= 6:
        cutoff = 0.8
    cutoffArtists = findNearest(item=artistName, ilist=artists, num=1, cutoff=cutoff)
    if len(cutoffArtists) > 0:
        for artist in cutoffArtists:
            mdata = getMusicData("Artist", artist)
            saveArtistData["Near"][artistName] = {"Rename": rename, "Discog": mdata, "Albums": artistAlbums}
            print('Near','\t',mdata.shape,'\t',artist,'\t(',artistName,')')
            done = True
    else:
        cutoffArtists = findNearest(item=artistName, ilist=artists, num=1, cutoff=cutoff - 0.15)
        if len(cutoffArtists) > 0:
            for artist in cutoffArtists:
                mdata = getMusicData("Artist", artist)
                saveArtistData["Near"][artistName] = {"Rename": rename, "Discog": mdata, "Albums": artistAlbums}
                print('Near','\t',mdata.shape,'\t',artist,'\t(',artistName,')')
                done = True

        
    if done:
        continue
        
    print('\t','\t','\t',artistName)
    saveArtistData["None"][artistName] = {"Rename": rename, "Discog": None, "Albums": artistAlbums}
    continue
    
    
    if artistName == "Anorexia Nervosa (2)":
        print(artistAlbums)
        1/0
    continue
    for albumName,albumVals in artistAlbums.items():
        print("\t",albumName,len(albumVals))    
    break
#albumData = {}

Match	 (1, 10) 	 '68 Comeback
Match	 (1, 10) 	 '68 Comeback Special
Match	 (1, 10) 	 *NSYNC
Match	 (1, 10) 	 :wumpscut:
Match	 (1, 10) 	 1.6 Band
Match	 (1, 10) 	 10,000 Maniacs
Match	 (1, 10) 	 1000 Clowns
Match	 (1, 10) 	 10cc
Match	 (1, 10) 	 112
Match	 (1, 10) 	 1/2 Japanese
Match	 (1, 10) 	 12 Lb. Test
Match	 (1, 10) 	 16
Match	 (1, 10) 	 2 Unlimited
Match	 (1, 10) 	 25 Ta Life
Match	 (1, 10) 	 2Pac
	 	 	 2Pac & Outlawz
Match	 (1, 10) 	 3 Doors Down
Match	 (1, 10) 	 3-Way Cum
Match	 (1, 10) 	 311
Odd	 (1, 10) 	 38 Special (2)
Match	 (1, 10) 	 3OH!3
Match	 (2, 10) 	 3T
Match	 (1, 10) 	 4 Non Blondes
Match	 (1, 10) 	 40 Grit
Match	 (1, 10) 	 454 Big Block
Match	 (1, 10) 	 5 Seconds Of Summer
Match	 (1, 10) 	 50 Cent
Match	 (1, 10) 	 59 Times The Pain
Match	 (4, 10) 	 666
Match	 (1, 10) 	 69 Boyz
Match	 (1, 10) 	 69 Tribe
Match	 (1, 10) 	 7 Angels 7 Plagues
Match	 (1, 10) 	 7000 Dying Rats
Match	 (1, 10) 	 764-Hero
Match	 (1, 10) 	 9 Shocks Terror
Match	 (1, 10) 	 9-Iron
Match	 (1, 1

	 	 	 Ben Webster & Oscar Peterson
Match	 (1, 10) 	 Benett Rogers
Match	 (1, 10) 	 Beniamino Gigli
Match	 (1, 10) 	 Benjamin B.
Match	 (1, 10) 	 Benny Goodman
Match	 (1, 10) 	 Bent Leg Fatima
Near 	 (1, 10) 	 BenÃ¼mb 	( Benümb )
Match	 (1, 10) 	 Berlin
Match	 (1, 10) 	 Bert Jansch
Match	 (1, 10) 	 Bertrand Burgalat
Match	 (1, 10) 	 Beseech
Match	 (1, 10) 	 Bessie Smith
Match	 (1, 10) 	 Beth Capper
Match	 (1, 10) 	 Bethzaida
Match	 (1, 10) 	 Bette Midler
Match	 (1, 10) 	 Better Than Ezra
Match	 (1, 10) 	 Beulah
Near 	 (1, 10) 	 BeyoncÃ© 	( Beyoncé )
Match	 (1, 10) 	 Beyond Dawn
Match	 (1, 10) 	 Beyond The Embrace
Near 	 (1, 10) 	 BÃ©la Fleck & The Flecktones 	( Béla Fleck & The Flecktones )
Odd	 (1, 10) 	 Bifrost (4)
Match	 (1, 10) 	 Big Ass Truck
Match	 (1, 10) 	 Big Bad Johns
Match	 (1, 10) 	 Big Boi
Match	 (1, 10) 	 Big Brother & The Holding Company
Match	 (1, 10) 	 Big Country
Match	 (1, 10) 	 Big D And The Kids Table
Match	 (1, 10) 	 Big Electric Cat
Match	 (1, 10) 	 Big Foot Chest

Match	 (2, 10) 	 Cathedral
Match	 (1, 10) 	 Cattle Decapitation
Match	 (1, 10) 	 Cause For Alarm
Match	 (1, 10) 	 Caustic Resin
Match	 (1, 10) 	 Cee-Lo
Match	 (3, 10) 	 Celebrity
Match	 (1, 10) 	 Celebrity Skin
Match	 (1, 10) 	 Celtic Frost
Match	 (1, 10) 	 Central Falls
Match	 (1, 10) 	 Centro-Matic
Match	 (1, 10) 	 Cephalic Carnage
Match	 (1, 10) 	 Cerberus Shoal
Match	 (1, 10) 	 Cerebros Exprimidos
Match	 (1, 10) 	 cEvin Key
Match	 (1, 10) 	 Céline Dion
	 	 	 Céline Dion & R. Kelly
Match	 (1, 10) 	 Chad Crouch
Match	 (1, 10) 	 Chad Kroeger
Odd	 (1, 10) 	 Chamberlain (2)
Match	 (1, 10) 	 Chan Romero
Odd	 (1, 10) 	 Channel 3 (2)
Match	 (1, 10) 	 Chanticleer
Match	 (1, 10) 	 Chaotic Dischord
Match	 (1, 10) 	 Charlie Barnet
Match	 (1, 10) 	 Charlie Musselwhite
Match	 (3, 10) 	 Charlie Parker
Match	 (1, 10) 	 Charlie Sexton
Match	 (1, 10) 	 Charm Pops
Match	 (1, 10) 	 Chavez
Match	 (1, 10) 	 Cheap Trick
Match	 (1, 10) 	 Cheater Slicks
Match	 (1, 10) 	 Cheer-Accident
Match	 (1, 10) 	 Chel

Match	 (3, 10) 	 Death By Chocolate
Match	 (1, 10) 	 Death By Stereo
Match	 (1, 10) 	 Death Cab For Cutie
Match	 (1, 10) 	 Death Ride 69
Odd	 (1, 10) 	 Death Threat (2)
Match	 (2, 10) 	 Deathwitch
Match	 (1, 10) 	 Decapitated
Match	 (1, 10) 	 Deceased
Odd	 (1, 10) 	 December (2)
Match	 (1, 10) 	 December Wolves
Match	 (1, 10) 	 Decoded Feedback
Match	 (2, 10) 	 Deep Forest
Match	 (1, 10) 	 Deep Purple
Match	 (1, 10) 	 Deepskin
Match	 (1, 10) 	 Def Leppard
Odd	 (1, 10) 	 Defiance (2)
Match	 (1, 10) 	 Defiled
Match	 (1, 10) 	 Defleshed
Match	 (1, 10) 	 Delta-Cross Band
Match	 (1, 10) 	 Demolition Doll Rods
Match	 (1, 10) 	 Denali
Match	 (1, 10) 	 Denata
Match	 (1, 10) 	 Denis Leary
Match	 (1, 10) 	 Dennis McGee
Match	 (1, 10) 	 Depeche Mode
Match	 (1, 10) 	 Descendents
Match	 (7, 10) 	 Desekrator
Match	 (1, 10) 	 Despise You
Match	 (1, 10) 	 Despite
Match	 (1, 10) 	 Destiny's Child
Match	 (1, 10) 	 Destiny's End
Match	 (1, 10) 	 Destroy All Monsters
Near 	 (1, 10) 	 DestrÃ¶yer 666 	( Des

Match	 (1, 10) 	 Ether Hogg
	 	 	 Eugene Chadbourne & Evan Johns
Match	 (1, 10) 	 Euphone
Match	 (1, 10) 	 Eurythmics
Match	 (1, 10) 	 Evan Dando
Match	 (1, 10) 	 Evanescence
Match	 (1, 10) 	 Eve 6
Match	 (1, 10) 	 Everclear
Match	 (1, 10) 	 Everlast
Match	 (1, 10) 	 Everly Brothers
Match	 (1, 10) 	 Everready
Match	 (1, 10) 	 Everything But The Girl
Match	 (1, 10) 	 Evil Beaver
Match	 (1, 10) 	 Evil Mothers
Match	 (1, 10) 	 Evils Toy
Match	 (2, 10) 	 Evoke
Match	 (1, 10) 	 Ewan MacColl
Match	 (1, 10) 	 Ex Number Five
Match	 (1, 10) 	 Ex-Fork
Match	 (1, 10) 	 Exclaim
Match	 (1, 10) 	 Excruciating Terror
Match	 (1, 10) 	 Exhumed
Match	 (1, 10) 	 EXP
Match	 (1, 10) 	 Experimental Aircraft
Match	 (1, 10) 	 Explosions In The Sky
Odd	 (1, 10) 	 Extreme (2)
Match	 (1, 10) 	 Extreme Noise Terror
Near 	 (1, 10) 	 Extreme Noise Terror 	( Extreme Noise Terror & Filthkick )
Match	 (2, 10) 	 Eye For An Eye
Match	 (1, 10) 	 Eyes Adrift
Match	 (1, 10) 	 F-Minus
Match	 (1, 10) 	 F.Y.P.
Match	 (1, 10) 

...

Near 	 (1, 10) 	 Mannie Fresh 	( Manie Fresh )
Match	 (1, 10) 	 Marc Pattison
Near 	 (1, 10) 	 Mark van Dale 	( Marc van Dale )
Odd	 (1, 10) 	 Marcus (58)
Match	 (4, 10) 	 Mario
Match	 (1, 10) 	 Mario Lanza
Match	 (1, 10) 	 Mario Lopez
Match	 (1, 10) 	 Mario Nascimbene
Match	 (1, 10) 	 Mark Dinning
Match	 (2, 10) 	 Mark Farina
	 	 	 Mark Johnson & Clawgrass
	 	 	 Mark Mancina & Trevor Rabin
Match	 (1, 10) 	 Mark Ronson
Match	 (1, 10) 	 Mark Snow
Match	 (1, 10) 	 Marky Mark
Match	 (1, 10) 	 Marlon Roudette
Match	 (8, 10) 	 Mars
	 	 	 Mars & D.N.A.
Match	 (1, 10) 	 Martika
Match	 (1, 10) 	 Martin Atkins
Odd	 (1, 10) 	 Martin Lawrence (2)
Near 	 (1, 10) 	 Mark Cutler 	( Martk Cutler )
Match	 (1, 10) 	 Marty Balin
Match	 (1, 10) 	 Marv Johnson
Match	 (1, 10) 	 Marva Whitney
Near 	 (1, 10) 	 Larry Darling 	( Marvy Darling )
	 	 	 Mary Costa & Bill Shirley
Match	 (1, 10) 	 Mary J. Blige
	 	 	 Mary Martin And Noel Coward
Match	 (2, 10) 	 Masako Hayashi
Match	 (6, 10) 	 Masquerade
Match	 (1, 1

	 	 	 Pigface-Lab Report
Match	 (2, 10) 	 Pigpen
	 	 	 Pigs N' Ratts
Match	 (2, 10) 	 Pillbox
Match	 (1, 10) 	 Pino Donaggio
	 	 	 Pitbull Ft. Ne-Yo, Afrojack & Nayer
Match	 (1, 10) 	 Pitch Shifter
Match	 (1, 10) 	 Planet Fuse
	 	 	 Plastic Angel & Raul De Chile
Match	 (1, 10) 	 Playahitty
	 	 	 Please Visit
Match	 (1, 10) 	 Pleasure P
Match	 (1, 10) 	 Plies
	 	 	 Plus Ones & Travoltas
Match	 (1, 10) 	 Pope John Paul II
Match	 (1, 10) 	 Poppis!
Near 	 (1, 10) 	 Porter Wagoner And Dolly Parton 	( Porter Wagoner & Dolly Parton )
Match	 (1, 10) 	 Postmorten
Near 	 (1, 10) 	 Power Age 	( Power Assume )
Match	 (7, 10) 	 Powerhouse
Odd	 (1, 10) 	 Praying Mantis (3)
Match	 (1, 10) 	 Precious Paris
Match	 (6, 10) 	 Pressure Point
Match	 (1, 10) 	 Preston Epps
Near 	 (1, 10) 	 The Princeton Footnotes 	( Princeton Footnotes )
Match	 (6, 10) 	 Problem
Match	 (2, 10) 	 Proclamation
Match	 (1, 10) 	 Professor Green
Odd	 (1, 10) 	 Project Z (2)
	 	 	 Pronto & Kalmani
	 	 	 Propellorheads & Shirley B

Odd	 (1, 10) 	 Slaughterhouse (7)
Near 	 (1, 10) 	 Sleaze Sisters 	( Sledge Sisters )
	 	 	 SledgeHammer x Twist
	 	 	 SlickF.B.
	 	 	 Slight Slappers And Short Hate Temper
Match	 (1, 10) 	 Slim K
Match	 (1, 10) 	 Slum Village
Match	 (1, 10) 	 SM-Trax
Match	 (6, 10) 	 Smith
Match	 (1, 10) 	 Smog
	 	 	 Smoke Dza & Action Bronson
Match	 (2, 10) 	 Smoove
Match	 (1, 10) 	 Snake Hips
Match	 (1, 10) 	 Sneaker Pimps
Match	 (20, 10) 	 Snoopy
Match	 (8, 10) 	 Solace
Match	 (1, 10) 	 Solomon Burke
Match	 (1, 10) 	 Sonia Sanchez
Match	 (3, 10) 	 Sonic Boom
Match	 (1, 10) 	 Sonic Dream Collective
Match	 (1, 10) 	 Sonny James
Match	 (1, 10) 	 Sonny Knight
Match	 (1, 10) 	 Southern Style Dj's
Near 	 (1, 10) 	 Spe 	( Spe$h )
	 	 	 Spiller vs Daft Punk
Match	 (1, 10) 	 Spooks
Match	 (1, 10) 	 Sprocket Wheel
	 	 	 Squeaky Burger
Odd	 (1, 10) 	 Squeeze (2)
	 	 	 SS77(Strik)
Match	 (4, 10) 	 St. Thomas
Match	 (1, 10) 	 Stack Bundles
Match	 (2, 10) 	 Stackabones
Odd	 (1, 10) 	 Star Wars (2)
Match	 (1, 10)

	 	 	 The Never Land Pirate Band
Match	 (1, 10) 	 The Neville Brothers
Match	 (2, 10) 	 The Noisettes
Near 	 (4, 10) 	 The World 	( The Old World )
Near 	 (1, 10) 	 The Old Glorys 	( The Old, Old Story )
Match	 (1, 10) 	 The Olympics
Match	 (3, 10) 	 The Originals
Match	 (1, 10) 	 The Osborne Brothers
Match	 (1, 10) 	 The Panoply Academy Glee Club
Match	 (1, 10) 	 The Paradons
Match	 (1, 10) 	 The Part-Time Losers
Match	 (1, 10) 	 The Penguins
	 	 	 The People The Poet
Odd	 (1, 10) 	 The Pits (2)
Match	 (3, 10) 	 The Pressure
Match	 (1, 10) 	 The Proclaimers
Odd	 (1, 10) 	 The Product (6)
Match	 (1, 10) 	 The Quin-Tones
Match	 (1, 10) 	 The Rays
Near 	 (1, 10) 	 The Rembrandts 	( The Rembrants )
Odd	 (1, 10) 	 The Reno Brothers (2)
Match	 (1, 10) 	 The Revels
Match	 (1, 10) 	 The Rich Kidz
Match	 (1, 10) 	 The Richard Hewson Orchestra
Match	 (1, 10) 	 The Rivingtons
Match	 (1, 10) 	 The Ronettes
Match	 (3, 10) 	 The Royal Teens
Match	 (1, 10) 	 The Safaris
	 	 	 The Saturdays Ft. Flo R

	 	 	 Yowda & Tracy T
Near 	 (1, 10) 	 Trap-A-Holics 	( Yowda & Trap-A-Holics )
Match	 (1, 10) 	 Yumi Arai
Match	 (1, 10) 	 Yung Joey
	 	 	 Yung Mazi & Jose Guapo
Match	 (1, 10) 	 Yves Deruyter
Near 	 (1, 10) 	 Zack Kent 	( Zack Kekona )
Match	 (1, 10) 	 Zarah Leander
Match	 (3, 10) 	 Zedd
Match	 (1, 10) 	 Zen Frisbee
Match	 (1, 10) 	 Zero Degrees Kelvin
Match	 (2, 10) 	 Zero Zero
Near 	 (1, 10) 	 Zyga 	( Zga )
Match	 (1, 10) 	 Zididada
Match	 (1, 10) 	 Zith
Near 	 (14, 10) 	 Zodiac 	( Zoediac )
Match	 (1, 10) 	 Zombies
Match	 (23, 10) 	 Zoom
Match	 (1, 10) 	 Zucchero
Match	 (1, 10) 	 Zymotix
	 	 	 ★L@♫BerT★-Eminem ft. Rihanna
Near 	 (2, 10) 	 Shots 	( 24 Shots )
	 	 	 Apply Pressure 3: Mayhem Music
	 	 	 Armand van Helden feat Duane H
	 	 	 Behind Da Bars
Match	 (1, 10) 	 Benja Styles
	 	 	 Big Mike & DJ Thoro
Near 	 (1, 10) 	 Society's Finest 	( Fifty's Finest )
	 	 	 First Class Cabin Vol. 2
	 	 	 Heartless Monster
Near 	 (1, 10) 	 HernÃ¡n CattÃ¡neo 	( Hernan Cattaneo )
	 	 	 Hernan

In [182]:
items = ["Björk", "Björn Afzelius", "Blue Öyster Cult", "Bjork", "Céline Dion", "Celine Dion"]
item  = "Céline Dion"
findNearest(item=item, ilist=items, num=1, cutoff=1.0)

['Céline Dion']

In [187]:
#retval = discdf[discdf[key] == artist]
item  = "Céline Dion"
discdf[discdf["Name"] == item]


,Ref,Name,Extra Artists,Genres,Styles,Albums Data,Known,Artist,Artist Num,Albums
19298,/artist/19298-Céline-Dion,Céline Dion,"[Celine Dion, Céline Dion, Celine]","[Pop, Rock, Electronic]","[Ballad, Vocal, Chanson]",{'90725': 'Beyond The Sun (12 Electronic Excur...,True,Céline Dion,NaN,"[Beyond The Sun (12 Electronic Excursions), Ne..."


In [195]:
for item in artistData.keys():
    if item.endswith(" Dion"):
        print(item,'\t\t',sum(discdf["Name"] == item))
        keys = ['NFC', 'NFKC', 'NFD', 'NFKD']
        for key in keys:
            my_str = normalize('NFC', item)
            print(item,key,'\t',sum(discdf["Name"] == my_str))
        print("")

Céline Dion 		 0
Céline Dion NFC 	 1
Céline Dion NFKC 	 1
Céline Dion NFD 	 0
Céline Dion NFKD 	 0

Aaron Neville & Celine Dion 		 0
Aaron Neville & Celine Dion NFC 	 0
Aaron Neville & Celine Dion NFKC 	 0
Aaron Neville & Celine Dion NFD 	 0
Aaron Neville & Celine Dion NFKD 	 0



In [184]:
items_load = list(artistData.keys())
item  = "Céline Dion"
print(findNearest(item=item, ilist=items, num=1, cutoff=1.0))
print(findNearest(item=item, ilist=items_load, num=1, cutoff=1.0))

['Céline Dion']
[]


In [144]:
my_str = "hello world"
my_str = "Céline Dion"
print(my_str,type(my_str))
my_str_as_bytes = str.encode(my_str)
print(my_str_as_bytes,type(my_str_as_bytes)) # ensure it is byte representation
my_decoded_str = my_str_as_bytes.decode()
print(my_decoded_str,type(my_decoded_str)) # ensure it is string representation

Céline Dion <class 'str'>
b'C\xc3\xa9line Dion' <class 'bytes'>
Céline Dion <class 'str'>


In [181]:
x = "Céline Dion"

from unicodedata import normalize
keys = ['NFC', 'NFKC', 'NFD', 'NFKD']
for key in keys:
    print(x,'<---',key)    
    print(findNearest(item=x, ilist=items, num=1, cutoff=1.0))
    my_str = normalize(key, x)
    print(findNearest(item=my_str, ilist=items, num=1, cutoff=1.0))
    print(my_str,type(my_str))
    my_str_as_bytes = str.encode(my_str, 'utf-8')
    print(my_str_as_bytes,type(my_str_as_bytes)) # ensure it is byte representation
    my_decoded_str = my_str_as_bytes.decode()
    print(my_decoded_str,type(my_decoded_str)) # ensure it is string representation
    print(findNearest(item=my_decoded_str, ilist=items, num=1, cutoff=1.0))
    print("")
    

Céline Dion <--- NFC
['Céline Dion']
['Céline Dion']
Céline Dion <class 'str'>
b'C\xc3\xa9line Dion' <class 'bytes'>
Céline Dion <class 'str'>
['Céline Dion']

Céline Dion <--- NFKC
['Céline Dion']
['Céline Dion']
Céline Dion <class 'str'>
b'C\xc3\xa9line Dion' <class 'bytes'>
Céline Dion <class 'str'>
['Céline Dion']

Céline Dion <--- NFD
['Céline Dion']
[]
Céline Dion <class 'str'>
b'Ce\xcc\x81line Dion' <class 'bytes'>
Céline Dion <class 'str'>
[]

Céline Dion <--- NFKD
['Céline Dion']
[]
Céline Dion <class 'str'>
b'Ce\xcc\x81line Dion' <class 'bytes'>
Céline Dion <class 'str'>
[]



In [150]:
for my_str in [x for x in saveArtistData["Near"].keys() if x.startswith("C") and x.endswith("n")]:
    print(my_str,type(my_str))
    my_str_as_bytes = str.encode(my_str, 'utf-8')
    print(my_str_as_bytes,type(my_str_as_bytes)) # ensure it is byte representation
    my_decoded_str = my_str_as_bytes.decode()
    print(my_decoded_str,type(my_decoded_str)) # ensure it is string representation
    print("")

Céline Dion <class 'str'>
b'Ce\xcc\x81line Dion' <class 'bytes'>
Céline Dion <class 'str'>

C.B Milton <class 'str'>
b'C.B Milton' <class 'bytes'>
C.B Milton <class 'str'>

Cee-Lo Green <class 'str'>
b'Cee-Lo Green' <class 'bytes'>
Cee-Lo Green <class 'str'>

Charlie Waller And The Country Gentlemen <class 'str'>
b'Charlie Waller And The Country Gentlemen' <class 'bytes'>
Charlie Waller And The Country Gentlemen <class 'str'>

Chinese Assassin <class 'str'>
b'Chinese Assassin' <class 'bytes'>
Chinese Assassin <class 'str'>



In [141]:
help(str.encode)

Help on method_descriptor:

encode(self, /, encoding='utf-8', errors='strict')
    Encode the string using the codec registered for encoding.
    
    encoding
      The encoding in which to encode the string.
    errors
      The error handling scheme to use for encoding errors.
      The default is 'strict' meaning that encoding errors raise a
      UnicodeEncodeError.  Other possible values are 'ignore', 'replace' and
      'xmlcharrefreplace' as well as any other name registered with
      codecs.register_error that can handle UnicodeEncodeErrors.



In [169]:
items

['Björk',
 'Björn Afzelius',
 'Blue Öyster Cult',
 'Bjork',
 'Céline Dion',
 'Celine Dion']

In [188]:
my_str = normalize("NFKC", item)
print(my_str)

for artistName in saveArtistData["Near"].keys():
    if artistName.endswith(" Dion"):
        print(sum(discdf["Name"] == item))
        
        break
        print(artistName,type(artistName),'\t',
              normalize("NFKC", artistName),'\t',
              str.encode(artistName, 'utf-8'),'\t',
              str.encode(item, 'utf-8'),'\t',
              str.encode(my_str, 'utf-8'),'\t',
              findNearest(item=artistName, ilist=items, num=1, cutoff=1.0),
              findNearest(item=my_str, ilist=items, num=1, cutoff=1.0))

Céline Dion
1


In [178]:
print(artistName,type(artistName),'\t',
      normalize("NFKC", artistName),'\tNearest -->',
      findNearest(item=artistName, ilist=items, num=1, cutoff=1.0))

Céline Dion <class 'str'> 	 Céline Dion 	Nearest --> []


In [166]:
print(item)

for artistName in saveArtistData["Near"].keys():
    if artistName.endswith(" Dion"):
        print(artistName,type(artistName),'\t',
              findNearest(item=artistName, ilist=items, num=1, cutoff=1.0),
              findNearest(item=item, ilist=items, num=1, cutoff=1.0))

Céline Dion
Céline Dion <class 'str'> 	 [] ['Céline Dion']


# Mark Soundtracks

In [215]:
from searchUtils import findDirsPattern, findAll
from fsUtils import moveDir
from fileUtils import getDirBasics
for dirval in findDirsPattern('/Volumes/Music/iTunes Compilation/iTunes Media/Music', pattern="OST"):
    if dirval.endswith("OST"):
        dst = " ".join([dirval[:-3].strip(), "Soundtrack"])
        albumName = " ".join([getDirBasics(dirval)[-1][:-3].strip(), "Soundtrack"])
        for val in findAll(dirval):
            for albumFile in findAll(val):
                m = mp3ID(albumFile)
                m.setAlbum(albumName)
                
        moveDir(dirval, dst)

# Near Matches

In [202]:
for i,(artistName,aData) in enumerate(saveArtistData["Near"].items()):
    print("{0: <3}/{1: <3}".format(i,len(saveArtistData["Near"])), end=" \t")
    rename = aData["Rename"]
    discog = aData["Discog"]
    albums = aData["Albums"]
    if discog is not None:
        matches = discog["Name"].values

        print('{0: <50}'.format(artistName),matches)
    else:
        print('{0: <50}{1: <8}{2}'.format(artistName,'x',len(albums)))

0  /653 	[spunge)                                           ['[spunge]']
1  /653 	Afrika Bambaataa & James Brown                     ['Afrika Bambaataa & Family']
2  /653 	Aksel Schiøtz                                      ['Aksel SchiÃ¸tz']
3  /653 	Alan Licht & Loren Mazzacane Connors               ['Loren Mazzacane Connors']
4  /653 	And Oceans                                         ['Me And Oceans']
5  /653 	Angelina Réaux                                     ['Angelina RÃ©aux']
6  /653 	Assück                                             ['AssÃ¼ck']
7  /653 	Bedřich Smetana                                    ['BedÅ\x99ich Smetana']
8  /653 	Ben Harper & The Blind Boys Of Alabama             ['The Blind Boys Of Alabama']
9  /653 	Benümb                                             ['BenÃ¼mb']
10 /653 	Beyoncé                                            ['BeyoncÃ©']
11 /653 	Béla Fleck & The Flecktones                        ['BÃ©la Fleck & The Flecktones']
12 /653 	Björk              

101/653 	Charlies Angels                                    ['Charlie Angel']
102/653 	Charlies Angels OST                                ['Charlie Angel']
103/653 	Cokie Roberts                                      ['Cole Roberts']
104/653 	Collaborations                                     ['The Collaboration' 'The Collaboration (2)']
105/653 	Coyote Ugly OST                                    ['Coyote Ugly']
106/653 	Crossroads OST                                     ['Crossroads' 'Crossroads (5)' 'Crossroads (9)']
107/653 	Dangerous Minds - Music From The Motion Picture    ['Music From The Motion Picture']
108/653 	Darlene Wilkinson                                  ['Dave Wilkinson']
109/653 	Dedication 2                                       ['Dedication']
110/653 	Dedication 3                                       ['Dedication']
111/653 	Dedication 4                                       ['Dedication']
112/653 	Desperado OST                                      ['Desperado' 'Desp

285/653 	Cee-Lo Green                                       ['Tee Green']
286/653 	Charley Musselwhite                                ['Charlie Musselwhite']
287/653 	Charlie Waller And The Country Gentlemen           ['Gene Bowers And The Country Gentleman']
288/653 	Chet's Speech                                      ['The Speech']
289/653 	Chinese Assassin                                   ['Canvas Assassin']
290/653 	Chip tha Ripper                                    ['Chip The Ripper']
291/653 	Chris Con Carne                                    ['Chris Corner' 'Chris Corner (2)']
292/653 	Chris Issak                                        ['Chris Isaak']
293/653 	Clarence Frogman Henry                             ['Clarence "Frogman" Henry']
294/653 	Coasters                                           ['Coaster (5)']
295/653 	Connells                                           ['The Connells']
296/653 	Corinne                                            ['Corinne (9)' 'Corinne' 'Corin

435/653 	James Taylor Quartet                               ['The James Taylor Quartet']
436/653 	James Zabiela & Sasha                              ['James Zabiela']
437/653 	Jan and Dean                                       ['Jan & Dean']
438/653 	Japanese Music                                     ['Jas Music']
439/653 	Jason Derülo                                       ['Jason Derulo']
440/653 	Jesper Dahlbäck                                    ['Jesper DahlbÃ¤ck']
441/653 	Jimmy King & King James Versio                     ['Jimmy King And The King James Version Band']
442/653 	Joe Bennett & The Sparkletones                     ['Joe Bennett And The Sparkletones']
443/653 	John Cougar                                        ['John Cough']
444/653 	John Rhyman                                        ['Jon Ryman']
445/653 	John Stirrat                                       ['John Stirratt']
446/653 	Johnny & The Hurricanes                            ['Johnny And The Hurricanes']
447/6

605/653 	UK & Whitelabel                                    ['Whitelabel (2)' 'Whitelabel' 'Whitelabel (3)']
606/653 	United Schach Corporation                          ['United Recording Corporation']
607/653 	Universiteskoret Lille MUKO                        ['Universitetskoret Lille MUKO']
608/653 	Unknown Album                                      ['Unknown Realm']
609/653 	Usurper II                                         ['Usurper (2)' 'Usurper' 'Usurper (6)']
610/653 	Uz Jsme Doma                                       ['UÅ¾ Jsme Doma']
611/653 	Vaughan, Stevie Ray &  Double Trouble              ['Stevie Ray Vaughan & Double Trouble']
612/653 	Vaughan, Stevie Ray & Double Trouble               ['Stevie Ray Vaughan & Double Trouble']
613/653 	Vinni Smith                                        ['Vinnie Smith']
614/653 	Warp Brothers vs Aquaqen                           ['Wrap Brother VS Aquagen']
615/653 	Wayne, Jon                                         ['Wayne Jones (2)' 'Wayn

# Odd Names

In [201]:
for artistName,aData in saveArtistData["Odd"].items():
    rename = aData["Rename"]
    discog = aData["Discog"]
    albums = aData["Albums"]
    if discog is not None:
        matches = discog["Name"].values
        
        print('{0: <50}'.format(artistName),matches)
    else:
        print('{0: <50}{1: <8}{2}'.format(artistName,'x',len(albums)))

38 Special (2)                                     ['38 Special (2)']
Abominator (2)                                     ['Abominator (2)']
Abscess (2)                                        ['Abscess (2)']
Absolution (2)                                     ['Absolution (2)']
Adele (3)                                          ['Adele (3)']
Alan Jackson (2)                                   ['Alan Jackson (2)']
Aloha (2)                                          ['Aloha (2)']
Anacrusis (2)                                      ['Anacrusis (2)']
Ancient (2)                                        ['Ancient (2)']
Andre Williams (2)                                 ['Andre Williams (2)']
Anorexia Nervosa (2)                               ['Anorexia Nervosa (2)']
Ashes (3)                                          ['Ashes (3)']
Aurora (12)                                        ['Aurora (12)']
Autograph (2)                                      ['Autograph (2)']
Avenger (6)                       

Rorschach (2)                                      ['Rorschach (2)']
Sabot (2)                                          ['Sabot (2)']
Sarge (2)                                          ['Sarge (2)']
Scarface (3)                                       ['Scarface (3)']
Scream (2)                                         ['Scream (2)']
Shai (3)                                           ['Shai (3)']
Shallow (2)                                        ['Shallow (2)']
Shelter (2)                                        ['Shelter (2)']
Shiner (5)                                         ['Shiner (5)']
Shutdown (2)                                       ['Shutdown (2)']
Silent Majority (3)                                ['Silent Majority (3)']
Skirt (3)                                          ['Skirt (3)']
Soil (2)                                           ['Soil (2)']
Solstice (2)                                       ['Solstice (2)']
Soul Asylum (2)                                    ['Soul Asylu

In [105]:
saveArtistData['Odd']['Venom (8)']['Discog']

,Ref,Name,Extra Artists,Genres,Styles,Albums Data,Known,Artist,Artist Num,Albums
251788,/artist/251788-Venom-8,Venom (8),[Venom (8)],"[Rock, Non-Music]","[Black Metal, Heavy Metal, Speed Metal]","{'213956': 'Possessed', '593990': 'Cast In Sto...",True,Venom,8.0,"[Possessed, Cast In Stone (Part Two), Buried A..."


In [125]:
from mp3id import mp3ID
mp3Vals = {}
for artistName,aData in saveArtistData['Odd'].items():
    print(artistName)
    discog = aData['Discog']
    artist = None
    if isinstance(discog, DataFrame):
        if discog.shape[0] == 1:
            artist = discog['Artist'][0] #.values
            
    if artist is None:
        continue
        
    for album,albumFiles in aData['Albums'].items():
        print('  ',album)
        for albumFile in albumFiles:
            print('\t',albumFile)
            m = mp3ID(albumFile)
            mArtist = m.getArtist()[0]
            if mArtist == artistName:
                print("\t\tArtist: {0} <--> {1}".format(mArtist,artist))
                m.setArtist(artist)

            mAlbumArtist = m.getAlbumArtist()[0]
            if mAlbumArtist == artistName:
                print("\t\tAlbumArtist: {0} <--> {1}".format(mAlbumArtist,artist))
                m.setAlbumArtist(artist)
            #mp3Vals[albumFile] = mid.getInfo()
            
        



38 Special (2)
   Flashback
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/38 Special (2)/Flashback/05 Caught Up in You.mp3
   Time Served
	 /Volumes/Music/iTunes Consolidate/iTunes Media/Music/38 Special (2)/Time Served/01 01 - Life Value.mp3
	 /Volumes/Music/iTunes Consolidate/iTunes Media/Music/38 Special (2)/Time Served/02 02 - Fire.mp3
	 /Volumes/Music/iTunes Consolidate/iTunes Media/Music/38 Special (2)/Time Served/03 03 - Room Full Of Money.mp3
	 /Volumes/Music/iTunes Consolidate/iTunes Media/Music/38 Special (2)/Time Served/04 04 - The Appeal Ft Chinx Drugz French Montana.mp3
	 /Volumes/Music/iTunes Consolidate/iTunes Media/Music/38 Special (2)/Time Served/05 05 - These 3 Words.mp3
	 /Volumes/Music/iTunes Consolidate/iTunes Media/Music/38 Special (2)/Time Served/06 06 - Gone.mp3
	 /Volumes/Music/iTunes Consolidate/iTunes Media/Music/38 Special (2)/Time Served/07 07 - Snow.mp3
	 /Volumes/Music/iTunes Consolidate/iTunes Media/Music/38 Special (2)/Time Served/08 08 - House Aro

		Artist: Abscess (2) <--> Abscess
		AlbumArtist: Abscess (2) <--> Abscess
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Abscess (2)/Tormented/06 Street Trash.mp3
		Artist: Abscess (2) <--> Abscess
		AlbumArtist: Abscess (2) <--> Abscess
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Abscess (2)/Tormented/07 Halo Of Disease.mp3
		Artist: Abscess (2) <--> Abscess
		AlbumArtist: Abscess (2) <--> Abscess
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Abscess (2)/Tormented/08 Scratching At The Coffin.mp3
		Artist: Abscess (2) <--> Abscess
		AlbumArtist: Abscess (2) <--> Abscess
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Abscess (2)/Tormented/09 Ratbag.mp3
		Artist: Abscess (2) <--> Abscess
		AlbumArtist: Abscess (2) <--> Abscess
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Abscess (2)/Tormented/10 Death Runs Red.mp3
		Artist: Abscess (2) <--> Abscess
		AlbumArtist: Abscess (2) <--> Abscess
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Abscess (2)/Tormented/1

		Artist: Absolution (2) <--> Absolution
		AlbumArtist: Absolution (2) <--> Absolution
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Absolution (2)/Complete Discography/24 _Antidote.mp3
		Artist: Absolution (2) <--> Absolution
		AlbumArtist: Absolution (2) <--> Absolution
Adele (3)
   19
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/19/01 Daydreamer 1.mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/19/01 Daydreamer 2.mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/19/01 Daydreamer 3.mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/19/01 Daydreamer 4.mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/19/01 Daydreamer.mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/19/02 Best for Last 1.mp3
		AlbumArtist: Adele 

		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/iTunes Live From SoHo/02 Right As Rain (Live).mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/iTunes Live From SoHo/03 Make You Feel My Love (Live).mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/iTunes Live From SoHo/04 Melt My Heart to Stone (Live).mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/iTunes Live From SoHo/05 Hometown Glory (Live).mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/iTunes Live From SoHo/06 Chasing Pavements (Live).mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3)/iTunes Live From SoHo/07 Fool That I Am (Live).mp3
		AlbumArtist: Adele (3) <--> Adele
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Adele (3

	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Anacrusis (2)/Manic Impressions/07 What You Became.mp3
		Artist: Anacrusis (2) <--> Anacrusis
		AlbumArtist: Anacrusis (2) <--> Anacrusis
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Anacrusis (2)/Manic Impressions/08 Our Reunion.mp3
		Artist: Anacrusis (2) <--> Anacrusis
		AlbumArtist: Anacrusis (2) <--> Anacrusis
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Anacrusis (2)/Manic Impressions/09 Idle Hours.mp3
		Artist: Anacrusis (2) <--> Anacrusis
		AlbumArtist: Anacrusis (2) <--> Anacrusis
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Anacrusis (2)/Manic Impressions/10 Far Too Long.mp3
		Artist: Anacrusis (2) <--> Anacrusis
		AlbumArtist: Anacrusis (2) <--> Anacrusis
   Screams And Whispers
	 /Volumes/Music/iTunes Matched/iTunes Media/Music/Anacrusis (2)/Screams And Whispers/01 Sound The Alarm.mp3
		Artist: Anacrusis (2) <--> Anacrusis
		AlbumArtist: Anacrusis (2) <--> Anacrusis
	 /Volumes/Music/iTunes Matched/iTunes M

...

		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/2-12 Love Pledge and the Arena.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/2-13 Confrontation with Count Dooku and Finale.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/3-01 Star Wars and the Revenge of the Sith.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/3-0

		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/5-04 The Death Star  the Stormtroopers.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/5-05 Wookie Prisoner  Detention Block Ambush.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/5-06 Shootout in the Cell Bay  Dianoga.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/5

		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/8-08 The Emperor Arrives  the Death of Yoda  Obi-Wan's Revelation.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/8-09 Alliance Assembly.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collection/8-10 Shuttle Tydirium Approaches Endor.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Star Wars: The Ultimate Soundtrack Collect

		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Summon The Heroes/04 Ode To Zeus _ Canto Olympico _ Mikis Theodorakis.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Summon The Heroes/05 Javelin _ Michael Torke.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Summon The Heroes/06 Olympic Hymm _ Leonard Bernstein.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/Summon The Heroes/07 Festive Overture _ Dmitri Shostakovich.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams

		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
   The Patriot Soundtrack
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/The Patriot Soundtrack/01 The Patriot.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/The Patriot Soundtrack/03 To Charleston.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/The Patriot Soundtrack/04 The Colonial Cause.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volumes/Music/iTunes Classical/iTunes Media/Music/John Williams (4)/The Patriot Soundtrack/07 Preparing For Battle.mp3
		Artist: John Williams (4) <--> John Williams
		AlbumArtist: John Williams (4) <--> John Williams
	 /Volume

In [108]:
mp3Vals

{'/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Venom (8)/The Best Of Venom (1981-1994)/01 Angeldust.mp3': {'Version': None,
  'Artist': ['Venom (8)'],
  'AlbumArtist': ['Venom (8)'],
  'Album': ['The Best Of Venom (1981-1994)', ''],
  'Title': ['Angeldust', ''],
  'TrackNo': ['1', ''],
  'DiscNo': None,
  'Compilation': None,
  'Language': None,
  'Size': False},
 '/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Venom (8)/The Best Of Venom (1981-1994)/02 Raise The Dead.mp3': {'Version': None,
  'Artist': ['Venom (8)'],
  'AlbumArtist': ['Venom (8)'],
  'Album': ['The Best Of Venom (1981-1994)', ''],
  'Title': ['Raise The Dead', ''],
  'TrackNo': ['2', ''],
  'DiscNo': None,
  'Compilation': None,
  'Language': None,
  'Size': False},
 '/Volumes/Music/iTunes Consolidate/iTunes Media/Music/Venom (8)/The Best Of Venom (1981-1994)/03 Red Light Fever.mp3': {'Version': None,
  'Artist': ['Venom (8)'],
  'AlbumArtist': ['Venom (8)'],
  'Album': ['The Best Of Venom (1981-1994)', 

# Multi Artists

In [56]:
artistDB = disc.getArtistNameToIDData()
mulArts  = multiArtist(cutoff=0.9, discdata=artistDB, exact=False)

Loading data from /Volumes/Music/Discog/db/ArtistNameToID.p
  --> This file is 8.7MB.
Loading /Volumes/Music/Discog/db/ArtistNameToID.p


In [62]:
from pandas import read_excel

In [82]:
renameReplacements

{'ACDC': 'AC/DC',
 'Andre Previn': 'André Previn',
 'Ke$ha': 'Kesha',
 'Jason Derülo': 'Jason Derulo',
 'Andrew Dice Clay': 'Andrew "Dice" Clay',
 'Flemming Bamse Jørgensen': 'Flemming "Bamse" Jørgensen',
 '12 Japanese': '1/2 Japanese',
 'José Carreras': 'José Carreras',
 'Weird Al Yankovic': '"Weird Al" Yankovic',
 'Østkyst Hustlers': 'Østkyst Hustlers',
 'AC-DC': 'AC/DC',
 'BØRNS': 'BØRNS'}

In [79]:
findNearest(item="Andre Previn", ilist=artistDB.keys(), num=10, cutoff=0.7)

['André Previn',
 'Andrew Prinz',
 'Andrew Preen',
 'Andrew Pevny',
 'Andres Levin',
 'Andrea Prevignano',
 'Andrew Paresi',
 'Andrew Levins',
 'Andrew Levien',
 'Andrei Eremin']

In [78]:
for artistName,artistData in saveArtistData.items():
    found = artistData[0]
    if found is False:
        retval = ma.getArtistNames(artistName)
        if len(retval) > 1:
            artistNames = [k for k,v in retval.items() if v is not None]
            print("{0: <50}{1}".format(artistName,artistNames))
        else:
            print("{0: <50}{1}".format(artistName,artistData[0]))

12 Japanese                                       False
ACDC                                              False
Andrew Dice Clay                                  False
Béla Fleck & The Flecktones                      []
Elliott Sharp  Carbon                             False
Eminem & Elton John                               ['Elton John', 'Eminem']
Flemming Bamse Jørgensen                          False
Fred Wesley & The J.B.'s                          ['Fred Wesley', "The J.B.'s"]
John Trubee & Ugly Janitors Of America            ['John Trubee', 'Ugly Janitors Of America']
Johnny Cash & June Carter                         ['June Carter', 'Johnny Cash']
José Carreras                                    False
Københavns Drengekor                              False
Louis Armstrong & Duke Ellington                  ['Louis Armstrong', 'Duke Ellington']
Macklemore & Ryan Lewis                           ['Macklemore', 'Ryan Lewis']
NeitherNeither World                              False
Niko

Michael 5000 Watts                                False
Michael Jordon                                    False
Mickey & Silvia                                   ['Mickey']
Mighty Flashlight                                 False
Mission to Mars                                   False
Morrie Schwartz                                   False
N.W.H.                                            False
Nitty Scott, MC                                   []
Nothin Fancy                                      False
Nu Jersey Devil                                   False
NWH                                               False
Pat Bone                                          False
peck                                              False
Phil Philips                                      False
Porter Wagoner & Dolly Parton                     ['Dolly Parton', 'Porter Wagoner']
Princeton Footnotes                               False
Puke-A-Rama                                       False
Pulsdriver       

In [46]:
lvals = list(discdf["Artist"])
test  = "*NSYNC"
from searchUtils import findNearest
findNearest(item=test, ilist=lvals, num=1, cutoff=0.9)

['*NSYNC']

In [27]:
def getMusicData(artist):
    retval = musicdf[musicdf["Artist"] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None

In [10]:
artistIDtoName = disc.getArtistIDToNameData()
artistIDtoRefs  = disc.getArtistIDToRefData()
albumIDtoName = disc.getAlbumIDToNameData()
albumIDtoRef   = disc.getAlbumIDToRefData()

artistIDtoAlbumNames = disc.getArtistIDAlbumNames()
artistIDtoAlbumIDs = disc.getArtistIDAlbumIDs()

artistMetaData = disc.getAlbumArtistMetaData()

Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 8.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 10.3MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Loading data from /Volumes/Music/Discog/db/ArtistIDAlbumNames.p
  --> This file is 72.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDAlbumNames.p
Loading data from /Volumes/Music/Discog/db/ArtistIDAlbumIDs.p
  --> This file is 42.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDAlbumIDs.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 10.3MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 14.3MB.
Loading /Volumes/Music/Discog/db/AlbumIDToRef.p
Loading data from /Volumes/Music/Discog/db/AlbumArtistMetaData.p
  --> This file is 13.8MB.
Loading /Volumes/Music/Discog/db/AlbumArtistMetaData.p


In [57]:
colArtistIDtoName = disc.getCollectionIDToNameData()
colArtistIDtoRefs = disc.getCollectionIDToRefData()
colArtistReftoCnts = disc.getCollectionRefCountsData()

artistIDtoName = disc.getArtistIDToNameData()
artistIDtoRefs  = disc.getArtistIDToRefData()
albumIDtoName = disc.getAlbumIDToNameData()
albumIDtoRef   = disc.getAlbumIDToRefData()

artistIDtoAlbumNames = disc.getArtistIDAlbumNames()
artistIDtoAlbumIDs = disc.getArtistIDAlbumIDs()

artistMetaData = disc.getAlbumArtistMetaData()



def splitMetaData(x):
    retval = {}
    if isinstance(x, dict):
        for k,v in x.items():
            retval[k] = [z[0] for z in v.most_common(3)]
    else:
        retval = None
    return retval

from pandas import Series, DataFrame
sArtistToRef  = Series(artistIDtoRefs)
sArtistToName = Series(artistIDtoName)
sAlbumToRef   = Series(albumIDtoRef)
sAlbumToName  = Series(albumIDtoName)

sArtistToAlbums = Series(artistIDtoAlbumIDs)
sArtistToAlbumNames = Series(artistIDtoAlbumNames)
sArtistAlbums = Series([dict(zip(x, y)) for x,y in list(zip(sArtistToAlbums.values, sArtistToAlbumNames))], index=sArtistToAlbums.index)

sArtistMetaData = Series(artistMetaData)
sArtistMetaData = sArtistMetaData.apply(splitMetaData)


cols = ["Ref"]
discdf = DataFrame(sArtistToRef)
discdf.columns = cols
discdf = discdf.join(DataFrame(sArtistToName))
cols += ["Name"]
discdf.columns = cols
tmp = DataFrame(DataFrame(sArtistMetaData)[0].tolist())
tmp.index = sArtistMetaData.index
discdf = discdf.join(tmp)
cols += ["Extra Artists", "Genres", "Styles"]
discdf.columns = cols
discdf = discdf.join(DataFrame(sArtistAlbums))
cols += ["Albums"]
discdf.columns = cols




sColArtistToRef  = Series(colArtistIDtoRefs)
sColArtistToName = Series(colArtistIDtoName)
sColArtistRefToCnts = Series(colArtistReftoCnts)

cols = ["Ref"]
coldiscdf = DataFrame(sColArtistToRef)
coldiscdf.columns = cols
coldiscdf = coldiscdf.join(DataFrame(sColArtistToName))
cols += ["Name"]
coldiscdf.columns = cols

colrefdf = DataFrame(sColArtistRefToCnts)
colrefdf.columns = ["Counts"]
colrefdf.reset_index(inplace=True)
colrefdf.columns = ["Ref", "Counts"]
coldiscdf = coldiscdf.merge(colrefdf, on="Ref")
coldiscdf.index = DataFrame(sColArtistToRef).index

coldiscdf = coldiscdf.merge(discdf, on=["Ref", "Name"], how='left')
coldiscdf.index = DataFrame(sColArtistToRef).index

Loading data from /Volumes/Music/Discog/db/CollectionIDToName.p
  --> This file is 29.4MB.
Loading /Volumes/Music/Discog/db/CollectionIDToName.p
Loading data from /Volumes/Music/Discog/db/CollectionIDToRef.p
  --> This file is 34.8MB.
Loading /Volumes/Music/Discog/db/CollectionIDToRef.p
Loading data from /Volumes/Music/Discog/db/CollectionRefCounts.p
  --> This file is 27.1MB.
Loading /Volumes/Music/Discog/db/CollectionRefCounts.p


In [79]:
sColArtistToRef  = Series(colArtistIDtoRefs)
sColArtistToName = Series(colArtistIDtoName)
sColArtistRefToCnts = Series(colArtistReftoCnts)

cols = ["Ref"]
coldiscdf = DataFrame(sColArtistToRef)
coldiscdf.columns = cols
coldiscdf = coldiscdf.join(DataFrame(sColArtistToName))
cols += ["Name"]
coldiscdf.columns = cols

colrefdf = DataFrame(sColArtistRefToCnts)
colrefdf.columns = ["Counts"]
colrefdf.reset_index(inplace=True)
colrefdf.columns = ["Ref", "Counts"]
coldiscdf = coldiscdf.merge(colrefdf, on="Ref")
coldiscdf.index = DataFrame(sColArtistToRef).index

coldiscdf = coldiscdf.merge(discdf, on=["Ref", "Name"], how='left')
coldiscdf.index = DataFrame(sColArtistToRef).index

In [148]:
coldiscdf.to_pickle("ArtistDiscog.p")

In [147]:
coldiscdf[coldiscdf["Name"] == "Dave Matthews"]

,Ref,Name,Counts,Extra Artists,Genres,Styles,Albums
169117,/artist/169117-Dave-Matthews,Dave Matthews,125,[Dave Matthews],[Rock],"[Jazz-Rock, Folk Rock, Fusion]","{'390472': 'DMB Live Trax Vol. 23', '2541361':..."


,Ref,Name,Counts,Extra Artists,Genres,Styles,Albums
0,/artist/294746-Richard-Wagner,Wagner,2975,NaN,NaN,NaN,NaN
1,/artist/4756306-Edgardo-Donato-Y-Su-Orquesta-T...,Edgardo Donato Y Su Orquesta Típica,8,[Edgardo Donato Y Su Orquesta Típica],"[Classical, Folk, World, & Country]",[],"{'12296603': 'El Camion / Carton Ligador', '57..."
2,/artist/5224521-TrÃ­o-Cocomarola,TrÃ­o Cocomarola,2,NaN,NaN,NaN,NaN
3,/artist/4775262-Damasio-Esquivel-y-Su-Conjunto...,Damasio Esquivel y Su Conjunto Guaraní,9,[Damasio Esquivel y Su Conjunto Guaraní],"[Latin, Folk, World, & Country]","[Polka, Chamamé]",{'5747330': 'Pajaro Campana / Ah! Me Corriente...
4,/artist/392779-Carlos-Gardel,Carlos Gardel,606,[Carlos Gardel],"[Latin, Folk, World, & Country, Non-Music]","[Tango, Zamba, Ranchera]",{'1363649': 'Un Tropezón / Se Acabaron Los Ota...
5,/artist/254768-Benny-Goodman,Benny Goodman,2484,"[Benny Goodman, B.G.]","[Jazz, Stage & Screen, Classical]","[Big Band, Swing, Ragtime]","{'10679936': 'London Date', '8177684': 'Perfor..."
6,/artist/302676-David-Rose-amp-His-Orchestra,David Rose & His Orchestra,126,NaN,NaN,NaN,NaN
7,/artist/4369840-Enrique-Mora-Y-Su-Cuarteto-Típico,Enrique Mora Y Su Cuarteto Típico,9,[Enrique Mora Y Su Cuarteto Típico],[Latin],[Tango],"{'7868496': 'Las Tres De La MaÃ±ana / Loca', '..."
8,/artist/3745182-Rafael-Rossi-Y-Su-Trio,Rafael Rossi Y Su Trio,5,[Rafael Rossi Y Su Trio],[Latin],"[Ranchera, Tango]","{'7259655': 'En La Retreta / Zapallo Charqui',..."
9,/artist/777498-Nelly-Omar,Nelly Omar,24,[Nelly Omar],[Latin],[Tango],"{'7446999': 'Tristeza Criolla', '7203720': 'El..."


In [126]:
colrefdf

,Counts
/artist/294746-Richard-Wagner,2975
/artist/4756306-Edgardo-Donato-Y-Su-Orquesta-Típica,8
/artist/5224521-Trío-Cocomarola,10
/artist/4775262-Damasio-Esquivel-y-Su-Conjunto-Guaraní,9
/artist/392779-Carlos-Gardel,606
/artist/254768-Benny-Goodman,2484
/artist/302676-David-Rose-His-Orchestra,392
/artist/4369840-Enrique-Mora-Y-Su-Cuarteto-Típico,9
/artist/3745182-Rafael-Rossi-Y-Su-Trio,5
/artist/777498-Nelly-Omar,24


In [118]:
coldiscdf.shape

(1464418, 2)

In [116]:
coldiscdf.merge(colrefdf, on="Ref", how="left")

,Ref,Name,Counts
0,/artist/294746-Richard-Wagner,Wagner,0.999899
1,/artist/4756306-Edgardo-Donato-Y-Su-Orquesta-T...,Edgardo Donato Y Su Orquesta Típica,0.753015
2,/artist/5224521-TrÃ­o-Cocomarola,TrÃ­o Cocomarola,0.293645
3,/artist/4775262-Damasio-Esquivel-y-Su-Conjunto...,Damasio Esquivel y Su Conjunto Guaraní,0.783023
4,/artist/392779-Carlos-Gardel,Carlos Gardel,0.998711
5,/artist/254768-Benny-Goodman,Benny Goodman,0.999858
6,/artist/302676-David-Rose-amp-His-Orchestra,David Rose & His Orchestra,0.991018
7,/artist/4369840-Enrique-Mora-Y-Su-Cuarteto-Típico,Enrique Mora Y Su Cuarteto Típico,0.783023
8,/artist/3745182-Rafael-Rossi-Y-Su-Trio,Rafael Rossi Y Su Trio,0.668044
9,/artist/777498-Nelly-Omar,Nelly Omar,0.946474


In [85]:

coldiscdf.merge(sColArtistRefToCnts, on="")

/artist/294746-Richard-Wagner                             2975
/artist/4756306-Edgardo-Donato-Y-Su-Orquesta-Típica          8
/artist/5224521-Trío-Cocomarola                             10
/artist/4775262-Damasio-Esquivel-y-Su-Conjunto-Guaraní       9
/artist/392779-Carlos-Gardel                               606
/artist/254768-Benny-Goodman                              2484
/artist/302676-David-Rose-His-Orchestra                    392
/artist/4369840-Enrique-Mora-Y-Su-Cuarteto-Típico            9
/artist/3745182-Rafael-Rossi-Y-Su-Trio                       5
/artist/777498-Nelly-Omar                                   24
/artist/3311157-Enrique-Rodríguez-Y-Su-Orquesta             11
/artist/611365-Fernando-Torres                               7
/artist/1791401-Fernando-Albuerne                           15
/artist/777494-Osvaldo-Pugliese-Y-Su-Orquesta-Típica        52
/artist/2220793-Rodolfo-Biagi-Y-Su-Orquesta-Típica          18
/artist/3471390-Miguel-Caló-Y-Su-Orquesta-Típica       

In [71]:
discdf.shape

(459147, 6)

In [30]:
sArtistToAlbums     = Series(artistIDtoAlbumIDs)
sArtistToAlbumNames = Series(artistIDtoAlbumNames)

Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value
Loading data from /Volumes/Music/Discog/db/ArtistVariationNameToIDs.p
  --> This file is 20.0MB.
Loading /Volumes/Music/Discog/db/ArtistVariationNameToIDs.p


In [38]:
s = Series([dict(zip(x, y)) for x,y in list(zip(sArtistToAlbums.values, sArtistToAlbumNames))], index=sArtistToAlbums.index)

In [39]:
s

1000       {'856965': 'Fuse Presents Dave Clarke', '27412...
1000500                       {'1178875': 'Peak Controller'}
100200     {'2082992': 'Infinity EP (Instrumental)', '156...
1002000    {'124656': 'Blunted On Reality', '1002108': 'N...
1002600                                {'1180905': 'Quiero'}
1002900                         {'157011': 'Freedom Fields'}
1003900    {'157268': 'Untitled', '330170': 'Loca Motion ...
1005400    {'4313892': 'Project 64', '3689834': 'Lo Cell'...
1007400    {'649563': 'Monster', '807093': 'Yr Money Or Y...
1010100    {'7858352': 'Who's That?', '7342091': 'Music E...
1013700    {'3439153': 'Baby Don't Dance', '413068': 'The...
1014400    {'1103544': 'Visits Planet Earth', '1303527': ...
1016300    {'2285358': 'Gong / Tundra / SÃ¥ledes Saae Joh...
101700     {'367156': 'Swingin' With The Starr', '651273'...
102100     {'416600': 'Sex After 3 Years In A Submarine',...
1027500    {'3254474': 'Electric Romantic Orchestra', '10...
1028100    {'4502636': '

In [ ]:
fixes = {}
iDir  = None
debug = False
for ifile,iPathData in jzpathdata.items():
    iTagData = jztagdata[ifile]
    
    pathArtist = iPathData['pbArtist']
    
    maPathArtists     = ma.getArtistNames(pathArtist)
    maTagArtists      = ma.getArtistNames(iTagData['Artist'])
    maTagAlbumArtists = ma.getArtistNames(iTagData['AlbumArtist'])

    if maTagArtists == maTagAlbumArtists:
        if maTagAlbumArtists == maPathArtists:
            continue
            print("Looks good")
        else:
            if debug:
                print("\nDifferent Artists:")
                print("\tPath:  {0}".format(maPathArtists.keys()))
                print("\tTag:   {0}".format(maTagArtists.keys()))
                print("\tTagAA: {0}".format(maTagAlbumArtists.keys()))
            if iDir is None:
                iDir = ifile.split(pathArtist)[0]                
                fixes[iDir] = {}
                print("  Adding {0}".format(iDir))
            if fixes[iDir].get(pathArtist) is None:
                fixes[iDir][pathArtist] = {}
                print("\tAdding {0}".format(pathArtist))
            iData = ifile.split(pathArtist)[1][1:]
            fixes[iDir][pathArtist][iData] = {"Error": "Path", "Path": maPathArtists, "Artist": maTagArtists, "AlbumArtist": maTagAlbumArtists}
            print("\t  Adding {0}".format(iData))
    else:
        if debug:
            print("\nUnknown Artist to Discogs:")
            print("\tPath:  {0}".format(maPathArtists))
            print("\tTag:   {0}".format(maTagArtists))
            print("\tTagAA: {0}".format(maTagAlbumArtists))


        if iDir is None:
            iDir = ifile.split(pathArtist)[0]                
            fixes[iDir] = {}
            print("  Adding {0}".format(iDir))
        if fixes[iDir].get(pathArtist) is None:
            fixes[iDir][pathArtist] = {}
            print("\tAdding {0}".format(pathArtist))
        iData = ifile.split(pathArtist)[1][1:]
        fixes[iDir][pathArtist][iData] = {"Error": "Tag", "Path": maPathArtists, "Artist": maTagArtists, "AlbumArtist": maTagAlbumArtists}
        print("\t  Adding {0}".format(iData))


In [ ]:
saveFile(idata=fixes, ifile="fixes.yaml")

In [ ]:
disc = discogs()
bsdata = searchDiscogs("Duke Ellington with the Ron Collier Orchestra Collages", requireUS=True, release=True)

In [ ]:
from webUtils import getHTML
from artist import artist
#bsdata = getHTML(bsdata)

In [ ]:

h4s = bsdata.findAll("h4")
h5s = bsdata.findAll("h5")



for ih4,h4 in enumerate(h4s):
    print(h4)
    spans = h4.findAll("span")
    ref   = None
    if len(spans) == 0:
        ref = h4.find("a")
    else:
        ref = spans[0].find("a")

    if ref is None:
        continue

    try:
        href   = ref.attrs.get('href')
        name   = ref.text.strip()
    except:
        print("Could not get name/href from {0}".format(ref))
        continue


In [ ]:

ah5s

In [ ]:
artistData

In [ ]:
albumData  = [x for x in h4s if len(x.findAll('a')) > 0]
albumData  = [x.findAll('a') for x in albumData if not x.find('a').attrs['href'].startswith("/sell/")]

artistData = [x.findAll("a") for x in h5s if len(x.findAll('span')) > 0]
#artistData = [x.findAll("a") for x in artistData]

In [ ]:
                    baseURL = disc.discogURL
                    url = urllib.parse.urljoin(baseURL, href)
                    self.downloadAlbumURL(url, savename)
                

In [ ]:
def getRefName(ref):
    try:
        href   = ref.attrs.get('href')
        name   = ref.text.strip()
    except:
        href   = None
        name   = None
        
    return href,name

In [ ]:
artistData

In [ ]:
alb = album()

In [ ]:
baseURL = disc.discogURL

for (albums,artists) in zip(albumData,artistData):
    for album in albums:
        albumRef,albumName = getRefName(album)
        print(albumRef,'\t',albumName)
        try:
            albumID  = str(int(albumRef.split("/")[-1]))
        except:
            print("Could not get album ID from {0}".format(albumRef))
            albumID  = None
            
        print(albumID)
        url = urllib.parse.urljoin(baseURL, albumRef)
                
                
    for artist in artists:
        artistRef,artistName = getRefName(artist)        
        print(artistRef,'\t',artistName)
        
    

In [ ]:
album

In [ ]:
from searchUtils import findDirsPattern, findWalkExt
from fileUtils import getBaseFilename, getFileBasics
from ioUtils import saveFile, getFile
from timeUtils import clock, elapsed
from musicPath import pathBasics


##############################################################################################################################
# My Music
##############################################################################################################################
class musicFileAnalyzer(musicBase):
    def __init__(self, debug=False):
        self.name = "myfileanalyzer"
        musicBase.__init__(self, debug=debug)
        
        self.pb      = pathBasics()
        self.pathDBs = self.getMusicPathDBs()
        self.tagDBs  = self.getMusicTagDBs()

In [ ]:
mfa = musicFileAnalyzer()

In [ ]:
from fileUtils import getDirname
from collections import Counter

artistPathData = {}
albumPathData  = {}

for cls in ma.getMusicClasses():
    pathData = ma.getMusicPathDB(cls)
    tagData  = ma.getMusicTagDB(cls)
    
    start, cmt = clock("  Checking {0} mp3s".format(len(pathData)))
    for mp3,pb in pathData.items():
        
        artist = pb["pbArtist"]
        if artist is not None:
            if artistPathData.get(artist) is None:
                artistPathData[artist] = {}
        else:
            if artistPathData.get(artist) is None:
                artistPathData[artist] = {}

            
        album = pb["pbAlbum"]
        if album is not None:
            if artistPathData[artist].get(album) is None:
                artistPathData[artist][album] = {}
        else:
            if artistPathData[artist].get(album) is None:
                artistPathData[artist][album] = {}

            
        cls = pb["pbClass"]
        if album is not None:
            if artistPathData[artist][album].get(cls) is None:
                artistPathData[artist][album][cls] = {}
        else:
            if artistPathData[artist][album].get(cls) is None:
                artistPathData[artist][album][cls] = {}

                
        dirval = getDirname(mp3)
        
        if artistPathData[artist][album][cls].get(dirval) is None:
            artistPathData[artist][album][cls][dirval] = []
        artistPathData[artist][album][cls][dirval].append([pb, tagData[mp3]])
    elapsed(start, cmt)
    print("")
    
    
savename = setFile(ma.getDBDir(), "ArtistAlbumTagData.p")
saveFile(ifile=savename, idata=artistPathData, debug=True)


In [ ]:
savename = setFile(ma.getDBDir(), "ArtistAlbumTagData.p")
artistPathData = getFile(savename)

In [ ]:
nArt = 0
for artist,artistData in artistPathData.items():
    nArt += 1
    if nArt == 10:
        break
    for album,albumData in artistData.items():
        for cls,clsData in albumData.items():
            for dirval,dirvalData in clsData.items():
                
                dval = dirval.replace(ma.getMusicDir(), "")
                
                nFiles = len(dirvalData)
                partist = artist
                if len(artist) >= 40:
                    partist = "{0}...".format(artist[:36])
                palbum = album
                if len(album) >= 40:
                    palbum = "{0}...".format(album[:36])

                print("{0: <40}{1: <40}{2: <20}{3: <5}{4}".format(partist,palbum,cls,nFiles,dirval))

In [ ]:
artistPathData

In [ ]:
for ifile in ma.flists:
    tagfile  = ifile.replace(".p", "-Tags.p")
    pathfile = ifile.replace(".p", "-Paths.p")

    fileData = getFile(ifile)
    tagData  = getFile(tagfile)
    pathData = getFile(pathfile)

In [ ]:
from searchUtils import findExt
files = findExt(basedir=ma.getDBDir(), ext="*.p")
files = [x for x in files if sum([y in x for y in ["Tags.p", "Paths.p"]]) == 0]
files

In [ ]:
import os, sys
def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts

In [ ]:
os.path.split("24 Shots/01 Intro (Power Of The Dollar)/dfd")

In [ ]:
idir

In [ ]:
files[-2]

In [ ]:
ifile

In [ ]:
filename

In [ ]:
dirval

In [ ]:
artist

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()

In [ ]:
import argparse as ap
from os import getcwd
from mp3id import mp3ID
mid = mp3ID(ifile)
minfo = mid.getInfo()
from glob import glob
from fsUtils import isFile
from fileUtils import getExt
from searchUtils import findWalkExt, findWalkPattern
from os import listdir, walk

class mp3Tagger:
    def __init__(self, disc=None):
        self.name = "mp3tagger"
        self.cwd  = getcwd()
        
        self.infos = {}
        self.ma       = multiArtist(cutoff=0.9, discdata=None, exact=False)

        
        if disc is not None:
            self.setDiscogs(disc)
        
        self.exts = mp3ID().mp3exts
        

    def setDiscogs(self, disc):
        artistDB = disc.getArtistVariationNameToIDsData()
        self.setArtistsDB(artistDB=artistDB)
        
    def setArtistsDB(self, artistDB, cutoff=0.9):
        self.ma       = multiArtist(cutoff=0.9, discdata=artistDB, exact=False)
        
        
    def getFiles(self, dirval):
        self.infos = {}
        files = findWalkPattern(basedir=dirval, pattern=".")
        files = [x for x in files if getExt(x) in self.exts]
        for i,ifile in enumerate(files):
            if self.infos.get(ifile) is None:
                mid = mp3ID(ifile)
                minfo = mid.getInfo()
                self.infos[ifile] = minfo
                
        
    def get(self):
        return self.infos
        
    def getInfo(self, dirval=None):
        if dirval is not None:
            self.getFiles(dirval)
            
        print("{0: <3}{1: <4}{2: <4}{3: <50}{4: <50}{5: <40}{6: <40}".format("#", "Trk", "Dsc", "Title", "Album", "Artist", "Album Artist"))

        i = 0
        for ifile,minfo in self.infos.items():
            print("{0: <3}".format(i+1), end="")

            trkno = minfo.get('TrackNo')
            if trkno is None:
                trkno = -1
            elif isinstance(trkno, list):
                if len(trkno) == 1:
                    trkno = trkno[0]
                else:
                    trkno = str(trkno)
            print("{0: <4}".format(trkno), end="")

            discno = minfo.get('DiscNo')
            if discno is None:
                discno = -1
            elif isinstance(discno, list):
                if len(discno) == 1:
                    discno = discno[0]
                else:
                    discno = str(discno)                
            print("{0: <4}".format(discno), end="")

            title = minfo.get('Title')
            if title is None:
                title = "?"
            elif isinstance(title, list):
                if len(title) == 1:
                    title = title[0]
                else:
                    title = str(title)
            print("{0: <50}".format(title), end="")

            album = minfo.get('Album')
            if album is None:
                album = "?"
            elif isinstance(album, list):
                if len(album) == 1:
                    album = album[0]
                else:
                    album = str(album)
            print("{0: <50}".format(album), end="")

            artist = minfo.get('Artist')
            if artist is None:
                artist = "?"
            elif isinstance(artist, list):
                if len(artist) == 1:
                    artist = artist[0]
                else:
                    artist = str(artist)
            print("{0: <40}".format(artist), end="")

            albartist = minfo.get('AlbumArtist')
            if albartist is None:
                albartist = "?"
            elif isinstance(albartist, list):
                if len(albartist) == 1:
                    albartist = albartist[0]
                else:
                    albartist = str(albartist)
            print("{0: <40}".format(albartist), end="")
            print("")
            
            i += 1
            
    

    def getAlbumArtists(self):
        return [x.get("AlbumArtist") for ifile,x in self.infos.items()]

In [ ]:
mt = mp3Tagger(disc=disc)

In [ ]:
mt.getInfo(dirval="/Volumes/Music/iTunes Jazz/iTunes Media/Music/Miles Davis & Quincy Jones")

In [ ]:
from ioUtils import saveFile
retvals = {}
for ifile,minfo in mt.get().items():
    retvals[ifile] = {}

    for key in ["Artist", "AlbumArtist"]:
        val = minfo.get(key)
        if isinstance(val, list):
            if len(val) == 1:
                retval = mt.ma.getArtistNames(val[0])
                if len(retval) > 1:
                    artistNames = [k for k,v in retval.items() if v is not None]
                    retvals[ifile][key] = artistNames
                    
saveFile(idata=retvals, ifile="tmp.yaml")

In [ ]:
retvals

In [ ]:
tmp=[['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones'],
 ['Miles Davis & Quincy Jones']]

In [ ]:
m.getDBDir

In [ ]:
idir="/Volumes/Music/iTunes Jazz/iTunes Media/Music/Miles Davis & Quincy Jones


from random import shuffle
print("There are {0} chart artists".format(len(chartArtists)))
shuffle(chartArtists)
for i,artist in enumerate(chartArtists):
    if prevs.get(artist) is not None:
        continue
    retval = ma.getArtistNames(artist)